### como temos poucos dados testes, por termos poucos dados no geral, os accuracy oscilam muito

### modelos


- ~~Logistic Regression (non-regularized)~~

- ~~Logistic Regression (regularized)~~

    - ~~using ridge~~

    - ~~using lasso~~


- ~~Redes Neuronais (não regularizada)~~

- ~~Redes Neuronais (regularizada)~~

- ~~Support Vector Classification (SVC) (nao regularizado)~~

- ~~Support Vector Classification (SVC) (regularizado)~~

In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import datasets
from sklearn.svm import SVC

In [90]:
stats = {}

data_file = '02balanced.csv'
data_file = '04onlyFatig.csv'
data_file = "05smote.csv"
data_file = "06smote.csv"


stats['data_file'] = data_file
data = pd.read_csv(data_file)
data.sample(5)

,Name,Date,Appetite,SleepInd,StSpirit,pEffort,uaI,uaI(-1),DeltaWt (%),DeltaHrtwo,FatigInd,Sex_F
434,272.620004,1572024153,5.724001,-2.362,4.275999,4.827998,1.6356,2.12796,-0.006546,15.928018,Risk/Caution,0
262,275.000000,1581811200,3.000000,-5.000,3.000000,1.000000,0.0000,2.31000,0.000000,-42.000000,Optimal,0
531,274.000000,1571443200,5.000000,-4.000,4.000000,5.000000,1.7800,2.18000,0.007463,34.000000,Risk/Caution,0
541,270.000000,1588550400,6.000000,-7.000,5.000000,5.000000,2.0000,0.00000,0.000000,18.000000,Risk/Caution,1
645,267.000000,1580947200,6.000000,-4.500,3.000000,5.000000,2.2500,2.48000,-0.008403,-18.000000,Risk/Caution,1


In [91]:
def standardize_numeric_columns(df):
    # Separate numeric and non-numeric columns
    numeric_cols = ['Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                    'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']

    # Initialize StandardScaler
    scaler = StandardScaler()

    # Standardize only the numeric columns
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    # Display the resulting DataFrame
    return df 

df = standardize_numeric_columns(data)

In [92]:
def logistical_regression(df, sex="all", only_accuracy=False):
       """regressao logistica apresentada não está regularizada"""
       if sex=="all":
              filtered_df = df
       elif sex=="F":
              filtered_df = df[df["Sex_F"] > 0.5]
       elif sex=="M":
              filtered_df = df[df["Sex_F"] < 0.5]
       else:
              raise ValueError("Invalid")


       X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
              'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
       y = filtered_df['FatigInd']

       X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
       # Initialize the model
       model = LogisticRegression()

       # Train the model
       model.fit(X_train, y_train)

       # Make predictions
       y_pred = model.predict(X_test)
       print("Accuracy")
       print(accuracy_score(y_test, y_pred))


       # Display confusion matrix
       print("Confusion Matrix:")
       print(confusion_matrix(y_test, y_pred))

       """
       metrics = {
              "Logistic Regression": {
                     "Accuracy": accuracy_score(y_test, y_pred),
                     "F1 Score": f1_score(y_test, y_pred, average='weighted'),
                     "Precision": precision_score(y_test, y_pred, average='weighted'),
                     "Recall": recall_score(y_test, y_pred, average='weighted')
              }}
       print(metrics)
       """

       if not only_accuracy:
              # Classification report for precision, recall, and F1-score
              print("Classification Report:")
              print(classification_report(y_test, y_pred))
              # Display coefficients of the model (for each feature)
              print("Model Coefficients:", model.coef_)
              print("Intercept:", model.intercept_)

       return accuracy_score(y_test, y_pred)

a = logistical_regression(df, sex="all", only_accuracy=True)
b = logistical_regression(df, sex="F", only_accuracy=True)
c = logistical_regression(df, sex="M", only_accuracy=True)

stats['logistic_regression_all'] = a
stats['logistic_regression_F'] = b
stats['logistic_regression_M'] = c

Accuracy
0.5816326530612245
Confusion Matrix:
[[19  8  3]
 [10 16 13]
 [ 2  5 22]]
Accuracy
0.6326530612244898
Confusion Matrix:
[[11  3  2]
 [ 3  9  5]
 [ 0  5 11]]
Accuracy
0.7142857142857143
Confusion Matrix:
[[15  7  1]
 [ 1  8  2]
 [ 0  3 12]]


In [93]:
def logistical_regression_comPenalidade_RIDGE(df, sex="all", only_accuracy=False):
       """regressao logistica apresentada está regularizada:
       using Ridge (L2) regularization
       C parameter controls the strength of the regularization (with smaller values indicating stronger regularization
       # C is the inverse of regularization strength"""
       if sex=="all":
              filtered_df = df
       elif sex=="F":
              filtered_df = df[df["Sex_F"] > 0.5]
       elif sex=="M":
              filtered_df = df[df["Sex_F"] < 0.5]
       else:
              raise ValueError("Invalid")


       X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
              'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
       y = filtered_df['FatigInd']

       X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
       # Initialize the model
       model = LogisticRegression(penalty='l2', C=0.1)

       # Train the model
       model.fit(X_train, y_train)

       # Make predictions
       y_pred = model.predict(X_test)

       # Calculate accuracy
       accuracy = accuracy_score(y_test, y_pred)
       print(f'Accuracy: {accuracy}')
       #accuracy = accuracy_score(y_train, model.predict(X_train))
       #print(f'Accuracy: {accuracy}')

       # Display confusion matrix
       print("Confusion Matrix:")
       print(confusion_matrix(y_test, y_pred))

       if not only_accuracy:
              # Classification report for precision, recall, and F1-score
              print("Classification Report:")
              print(classification_report(y_test, y_pred))
              # Display coefficients of the model (for each feature)
              print("Model Coefficients:", model.coef_)
              print("Intercept:", model.intercept_)

       return accuracy

a = logistical_regression_comPenalidade_RIDGE(df, sex="all", only_accuracy=True)
b = logistical_regression_comPenalidade_RIDGE(df, sex="F", only_accuracy=True)
c = logistical_regression_comPenalidade_RIDGE(df, sex="M", only_accuracy=True)

stats['logistic_regression_comPenalidade_RIDGE_all'] = a
stats['logistic_regression_comPenalidade_RIDGE_F'] = b
stats['logistic_regression_comPenalidade_RIDGE_M'] = c

Accuracy: 0.6224489795918368
Confusion Matrix:
[[25  7  1]
 [12 16 13]
 [ 0  4 20]]
Accuracy: 0.7142857142857143
Confusion Matrix:
[[13  1  2]
 [ 4 10  3]
 [ 0  4 12]]
Accuracy: 0.6122448979591837
Confusion Matrix:
[[16  5  0]
 [ 3  0  7]
 [ 2  2 14]]


In [94]:
def logistical_regression_comPenalidade_LASSO(df, sex="all", only_accuracy=False):
       """regressao logistica apresentada está regularizada:
       using Lasso (L1) regularization
       C parameter controls the strength of the regularization (with smaller values indicating stronger regularization
       # C is the inverse of regularization strength"""
       if sex=="all":
              filtered_df = df
       elif sex=="F":
              filtered_df = df[df["Sex_F"] > 0.5]
       elif sex=="M":
              filtered_df = df[df["Sex_F"] < 0.5]
       else:
              raise ValueError("Invalid")


       X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
              'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
       y = filtered_df['FatigInd']

       X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
       # Initialize the model
       model = LogisticRegression(solver='liblinear', penalty='l1')

       # Train the model
       model.fit(X_train, y_train)

       # Make predictions
       y_pred = model.predict(X_test)

       # Calculate accuracy
       accuracy = accuracy_score(y_test, y_pred)
       print(f'Accuracy: {accuracy}')
       #accuracy = accuracy_score(y_train, model.predict(X_train))
       #print(f'Accuracy: {accuracy}')

       # Display confusion matrix
       print("Confusion Matrix:")
       print(confusion_matrix(y_test, y_pred))

       if not only_accuracy:
              # Classification report for precision, recall, and F1-score
              print("Classification Report:")
              print(classification_report(y_test, y_pred))
              # Display coefficients of the model (for each feature)
              print("Model Coefficients:", model.coef_)
              print("Intercept:", model.intercept_)
       
       return accuracy

a = logistical_regression_comPenalidade_LASSO(df, sex="all", only_accuracy=True)
b = logistical_regression_comPenalidade_LASSO(df, sex="F", only_accuracy=True)
c = logistical_regression_comPenalidade_LASSO(df, sex="M", only_accuracy=True)

stats['logistic_regression_comPenalidade_LASSO_all'] = a
stats['logistic_regression_comPenalidade_LASSO_F'] = b
stats['logistic_regression_comPenalidade_LASSO_M'] = c

Accuracy: 0.5306122448979592
Confusion Matrix:
[[18  2  6]
 [19  3 17]
 [ 1  1 31]]
Accuracy: 0.7755102040816326
Confusion Matrix:
[[14  4  1]
 [ 2 10  2]
 [ 1  1 14]]
Accuracy: 0.5306122448979592
Confusion Matrix:
[[11  2  1]
 [ 8  6  6]
 [ 2  4  9]]


In [95]:
def neural_network(df, sex="all"):
        """rede neural apresentada não está regularizada"""
        if sex=="all":
                filtered_df = df
        elif sex=="F":
                filtered_df = df[df["Sex_F"] > 0.5]
        elif sex=="M":
                filtered_df = df[df["Sex_F"] < 0.5]
        else:
                raise ValueError("Invalid")

        X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
        y = filtered_df['FatigInd']

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


        # Neural Network Model
        nn = MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=1000)
        nn.fit(X_train, y_train)
        nn_pred = nn.predict(X_test)
        nn_probs = nn.predict_proba(X_test)  # Probabilities for ROC-AUC

        metrics = {
                "Accuracy": accuracy_score(y_test, nn_pred),
                "F1 Score": f1_score(y_test, nn_pred, average='weighted'),
                "Precision": precision_score(y_test, nn_pred, average='weighted'),
                "Recall": recall_score(y_test, nn_pred, average='weighted'),
                "ROC-AUC": roc_auc_score(y_test, nn_probs, multi_class='ovr', average='weighted')
        }

        # Display metrics
        print(metrics)
        conf_matrix = confusion_matrix(y_test, nn_pred)
        print("Confusion Matrix:")
        print(conf_matrix)

        return accuracy_score(y_test, nn_pred)

a = neural_network(df, sex="all")
b = neural_network(df, sex="F")
c = neural_network(df, sex="M")

stats['neural_network_all'] = a
stats['neural_network_F'] = b
stats['neural_network_M'] = c

{'Accuracy': 0.7, 'F1 Score': np.float64(0.6992813884118232), 'Precision': np.float64(0.7036845507433742), 'Recall': np.float64(0.7), 'ROC-AUC': np.float64(0.8485711546451057)}
Confusion Matrix:
[[27 11  2]
 [ 6 29 12]
 [ 1  7 35]]
{'Accuracy': 0.49230769230769234, 'F1 Score': np.float64(0.4941724941724941), 'Precision': np.float64(0.49716949716949704), 'Recall': np.float64(0.49230769230769234), 'ROC-AUC': np.float64(0.6805804015106341)}
Confusion Matrix:
[[15  4  3]
 [ 5  7  8]
 [ 2 11 10]]
{'Accuracy': 0.6307692307692307, 'F1 Score': np.float64(0.6289122467727118), 'Precision': np.float64(0.6282910072383757), 'Recall': np.float64(0.6307692307692307), 'ROC-AUC': np.float64(0.797236941999626)}
Confusion Matrix:
[[13  6  2]
 [ 9 14  3]
 [ 0  4 14]]


In [96]:
def neural_network_regularizada(df, sex="all"):
        """rede neural apresentada está regularizada
        Ridge regularization) is applied through the alpha parameter."""
        if sex=="all":
                filtered_df = df
        elif sex=="F":
                filtered_df = df[df["Sex_F"] > 0.5]
        elif sex=="M":
                filtered_df = df[df["Sex_F"] < 0.5]
        else:
                raise ValueError("Invalid")

        X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
        y = filtered_df['FatigInd']

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


        # Neural Network Model
        nn = MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=250, alpha=0.01) #L2 regularization
        #(also known as Ridge regularization) is applied through the alpha parameter.
        nn.fit(X_train, y_train)
        nn_pred = nn.predict(X_test)
        nn_probs = nn.predict_proba(X_test)  # Probabilities for ROC-AUC

        metrics = {
                "Accuracy": accuracy_score(y_test, nn_pred),
                "F1 Score": f1_score(y_test, nn_pred, average='weighted'),
                "Precision": precision_score(y_test, nn_pred, average='weighted'),
                "Recall": recall_score(y_test, nn_pred, average='weighted'),
                "ROC-AUC": roc_auc_score(y_test, nn_probs, multi_class='ovr', average='weighted')
        }

        # Display metrics
        print(metrics)
        conf_matrix = confusion_matrix(y_test, nn_pred)
        print("Confusion Matrix:")
        print(conf_matrix)

        return accuracy_score(y_test, nn_pred)

a = neural_network_regularizada(df, sex="all")
b = neural_network_regularizada(df, sex="F")
c = neural_network_regularizada(df, sex="M")

stats['neural_network_regularizada_all'] = a
stats['neural_network_regularizada_F'] = b
stats['neural_network_regularizada_M'] = c

/opt/anaconda3/envs/FAA/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/FAA/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(


{'Accuracy': 0.7153846153846154, 'F1 Score': np.float64(0.7113816135699879), 'Precision': np.float64(0.7123339907955293), 'Recall': np.float64(0.7153846153846154), 'ROC-AUC': np.float64(0.8580691870204978)}
Confusion Matrix:
[[31  6  4]
 [10 22  8]
 [ 4  5 40]]
{'Accuracy': 0.5846153846153846, 'F1 Score': np.float64(0.5775836341482423), 'Precision': np.float64(0.5795458546030628), 'Recall': np.float64(0.5846153846153846), 'ROC-AUC': np.float64(0.758542097929747)}
Confusion Matrix:
[[14  1  3]
 [ 7 10  6]
 [ 2  8 14]]
{'Accuracy': 0.6307692307692307, 'F1 Score': np.float64(0.6257096303919046), 'Precision': np.float64(0.6287545787545787), 'Recall': np.float64(0.6307692307692307), 'ROC-AUC': np.float64(0.7968534236668824)}
Confusion Matrix:
[[12  5  1]
 [ 7 12  6]
 [ 2  3 17]]


/opt/anaconda3/envs/FAA/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(


In [97]:
def SVC_model_free(df, sex="all"):
        """svc model sem regularização (dps ler se é regularizacao e isso o termo correto)"""
        if sex=="all":
                filtered_df = df
        elif sex=="F":
                filtered_df = df[df["Sex_F"] > 0.5]
        elif sex=="M":
                filtered_df = df[df["Sex_F"] < 0.5]
        else:
                raise ValueError("Invalid")

        X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
        y = filtered_df['FatigInd']

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Create a Support Vector Classifier model
        svc_model = SVC(kernel='linear')  # You can change the kernel to 'rbf', 'poly', etc.

        # Train the model
        svc_model.fit(X_train, y_train)

        # Make predictions on the test set
        y_pred = svc_model.predict(X_test)

        # Evaluate the model
        print("Accuracy:", accuracy_score(y_test, y_pred))
        #print("Classification Report:\n", classification_report(y_test, y_pred))

        return accuracy_score(y_test, y_pred)

a = SVC_model_free(df, sex="all")
b = SVC_model_free(df, sex="F")
c = SVC_model_free(df, sex="M")

stats['SVC_model_free_all'] = a
stats['SVC_model_free_F'] = b
stats['SVC_model_free_M'] = c

Accuracy: 0.5846153846153846
Accuracy: 0.5692307692307692
Accuracy: 0.676923076923077


In [98]:
def SVC_model_regularizado(df, sex="all"):
        """svc model COM regularização (dps ler se é regularizacao e isso o termo correto)"""
        if sex=="all":
                filtered_df = df
        elif sex=="F":
                filtered_df = df[df["Sex_F"] > 0.5]
        elif sex=="M":
                filtered_df = df[df["Sex_F"] < 0.5]
        else:
                raise ValueError("Invalid")

        X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
        y = filtered_df['FatigInd']

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Create a Support Vector Classifier model
        svc_model = SVC(kernel='linear')  # You can change the kernel to 'rbf', 'poly', etc.
        svc_model_high_C = SVC(C=1000, kernel='linear')  # High penalty (focuses on reducing training error)
        svc_model_low_C = SVC(C=0.01, kernel='linear')  # Low penalty (allows more errors for larger margin)

        svc_model_high_C.fit(X_train, y_train)
        svc_model_low_C.fit(X_train, y_train)

        # Predictions
        y_pred_high_C = svc_model_high_C.predict(X_test)
        y_pred_low_C = svc_model_low_C.predict(X_test)

        # Evaluate accuracy
        print("Accuracy with high C (10):", accuracy_score(y_test, y_pred_high_C))
        print("Accuracy with low C (0.01):", accuracy_score(y_test, y_pred_low_C))

        return accuracy_score(y_test, y_pred_high_C), accuracy_score(y_test, y_pred_low_C)

a, a1 = SVC_model_regularizado(df, sex="all")
b, b2= SVC_model_regularizado(df, sex="F")
c, c3 = SVC_model_regularizado(df, sex="M")

stats['SVC_model_regularizado_all_high_C'] = a
stats['SVC_model_regularizado_all_low_C'] = a1
stats['SVC_model_regularizado_F_high_C'] = b
stats['SVC_model_regularizado_F_low_C'] = b2
stats['SVC_model_regularizado_M_high_C'] = c
stats['SVC_model_regularizado_M_low_C'] = c3

Accuracy with high C (10): 0.5076923076923077
Accuracy with low C (0.01): 0.49230769230769234
Accuracy with high C (10): 0.6615384615384615
Accuracy with low C (0.01): 0.6615384615384615
Accuracy with high C (10): 0.6153846153846154
Accuracy with low C (0.01): 0.47692307692307695


In [99]:
df = pd.read_excel('results.xlsx')
df = pd.concat([df, pd.DataFrame(stats, index=[0])], ignore_index=True)
df.to_excel('results.xlsx', index=False)